In [1]:
import pycromanager as pyc

In [2]:
from pycromanager import Core,Studio

In [3]:
core = Core()

In [4]:
core.get_position()

-146

In [5]:
brushless = "KBD101_Thor_Rotation"
core.get_position(brushless)

95.996

In [6]:
def ppm_to_thor(angle):
    return (-2*angle + 276)

def thor_to_ppm(kinesis_pos):
    return (276 - kinesis_pos) / 2

In [7]:
thor_to_ppm(33)

121.5

In [8]:
def set_angle(theta):
    theta = ppm_to_thor(theta)
    core.set_position(brushless,theta)
    core.wait_for_device(brushless)
set_angle(0)

In [9]:
from typing import Tuple,Dict,Any
import numpy as np
def snap_image() -> Tuple[np.ndarray, Dict[str, Any]]:
    core.snap_image()
    img = core.get_image()
    tags = {
        "Width": core.get_image_width(),
        "Height": core.get_image_height(),
        #"PixelType": core.get_pixel_type(),
        #"BytesPerPixel": core.get_bytes_per_pixel(),
        #"BitDepth": core.get_bitdepth(),
    }
    # Handle color cameras
    camera = core.get_property('Core','Camera')
    if camera in ["QCamera", "MicroPublisher6"]:
        if core.get_property(camera,'Color') == "ON":
            img = np.reshape(img, (tags["Height"], tags["Width"], 4))
            img = img[:, :, 0:3]  # Remove alpha
            img = np.flip(img, 2)  # Flip channels
        else:
            img = np.reshape(img, (tags["Height"], tags["Width"]))
    else:
        img = np.reshape(img, (tags["Height"], tags["Width"]))
    return img, tags


In [10]:
img,tags = snap_image()

Exception: java.lang.Exception: This operation can not be executed while sequence acquisition is running.
mmcorej.MMCoreJJNI.CMMCore_snapImage(Native Method)
mmcorej.CMMCore.snapImage(CMMCore.java:976)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
org.micromanager.pyjavaz.ZMQServer.runMethod(ZMQServer.java:431)
org.micromanager.pyjavaz.ZMQServer.parseAndExecuteCommand(ZMQServer.java:511)
org.micromanager.pyjavaz.ZMQServer.lambda$initialize$2(ZMQServer.java:121)
java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
java.util.concurrent.FutureTask.run(FutureTask.java:266)
java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
java.lang.Thread.r

In [11]:
import matplotlib.pyplot as plt

In [28]:
#plt.imshow(img,cmap='gray')
#plt.colorbar()

In [29]:
img.dtype

dtype('uint16')

In [30]:
img.shape

(2200, 2688)

In [12]:
import tifffile as tf

# pure_calibration

In [38]:
import os
os.getcwd()

'C:\\Users\\lociuser\\Codes\\smartpath'

In [13]:
from fastprogress import progress_bar

In [15]:
core.set_exposure(500)

In [16]:
for k in progress_bar([0,-5,5,90]):
    set_angle(k)
    if k == 90:
        core.set_exposure(10)
    else:
        core.set_exposure(500)
        
    theta = core.get_position(brushless)
    img,tags = snap_image()
    filename = rf'Micropublishertrial{k:.03f}_.tiff'
    tf.imwrite(filename,img.astype(np.uint16))    


In [54]:
for k in progress_bar(range(0,360),total=360):
    set_angle(k)
    theta = core.get_position(brushless)
    img,tags = snap_image()
    filename = rf'D:\UserData\MichealNelson\debeyer_data_calibration_20xPolp5NA\Calibration_angle{k:03}.tif'
    tf.imwrite(filename,img.astype(np.uint16))    
    with open('log.txt','a') as fid:
        fid.write(str([k, theta, img.mean()]))

# 3 mins for full 360 deg calibration with 150 ms exposure